# Fusion

Submission 2:
    - Tremor : Gradient Boosting Regression
    - Dyskinesia : Gradient Boosting Regression
    
Submission 4: 
    - Dyskinesia : Average of predictions 

In [6]:
import numpy as np
import os
from sklearn.ensemble import RandomForestClassifier
import csv
from sklearn.model_selection import cross_val_score
from sklearn.datasets import make_blobs
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.tree import DecisionTreeClassifier
from random import random
from sklearn.metrics import mean_squared_error
import pickle
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import GradientBoostingRegressor

%load_ext autoreload
%autoreload 2

from transform_data import *
from create_graphs import *
from local.get_final_scores_accuracy import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
# def final_score(mse_per_subjectid, nb_files_per_subject_id, training_or_test=''):
#     """
#     Compute the final score for the challenge given the arguments 
    
#     Keyword arguments:
#     - mse_per_subjectid: list of the mse per subject_id 
#     - nb_files_per_subject_id: list of the number of files per subject_id 
#     - training_or_test: string just for the purpose of printing the result 
#     """
#     numerator = np.sum([nb_file * mse for nb_file, mse in zip(np.sqrt(nb_files_per_subject_id), mse_per_subjectid)])
#     denominator = np.sum(np.sqrt(nb_files_per_subject_id))
#     #FIXME : Refactor so it's not printing by default 
#     print(training_or_test+'Final score : ', np.divide(numerator, denominator))
#     return np.divide(numerator, denominator)

In [3]:
# def get_final_score(vPredictions, vParID, vTrueLabels):
#     """
#     Compute the final score from the challenge and print the result
#     Keyword arguments: 
#     - vPredictions: Numpy array containing the predictions 
#     - VParID: list containing the subject_id 
#     - vTrueLabels: list containing the true labels 
#     """
#     mse_per_subjectId = []
#     nb_files_per_subjectId = []
#     for subject_id in np.unique(vParID):
#         #print('--- SUBJECT ID ', subject_id, '---')
#         vSubjectId = (vParID == subject_id)
#         #print(vSubjectId)
#         vPredictions_subjectId = vPredictions[vSubjectId]
#         vTrueLabels_subjectId = np.array(vTrueLabels)[vSubjectId]
#         mse_per_subjectId.append(mean_squared_error(vTrueLabels_subjectId, vPredictions_subjectId))
#         nb_files_per_subjectId.append(len(vPredictions_subjectId))
#         #print('MSE : ', mean_squared_error(vTrueLabels_subjectId, vPredictions_subjectId))
#     print('--- MSEscore ---')
#     final_score(mse_per_subjectId, nb_files_per_subjectId)


In [4]:
sDirOut = "/export/b19/mpgill/BeatPD_predictions_tryingout"

## Submission 2 - Gradient Boosting Regression

In [7]:
def RandomForest_fusion(lFilesPred, sFileLabels, iEstimators, sTypeLabel, nudge, rLR, iMD,iRS):
    # Read file labels (true labels)
    with open(sFileLabels, mode='r') as infile:
        reader = csv.reader(infile)
        dID = {rows[0]:rows[1] for rows in reader} #participant ID
    with open(sFileLabels, mode='r') as infile:
        reader = csv.reader(infile)
        dOnOff= {rows[0]:rows[2] for rows in reader} #on-off label
    with open(sFileLabels, mode='r') as infile:
        reader = csv.reader(infile)
        dDys={rows[0]:rows[3] for rows in reader} #dyskinesia label
    with open(sFileLabels, mode='r') as infile:
        reader = csv.reader(infile)
        dTrem={rows[0]:rows[4] for rows in reader} #tremor label
#mydict = dict((rows[0],rows[1]) for rows in reader)
    
    # Training-testing data
    iNumFiles=len(lFilesPred)
#mPredictions=[] #np.zeros((1,iNumFiles))
#mPredictions=np.asarray(mPredictions)
    vPredIter=np.zeros((1,iNumFiles))
    vLabels=[] #true label
    vParID=[] # participant ID
    lDicts=[] 


    for sFilePred in lFilesPred:
        with open(sFilePred, mode='r') as infile:
            reader = csv.reader(infile)
            dPred = {rows[0]:rows[1] for rows in reader} #Prediction from the different classifiers (ivec-svr, boost...)
        lDicts.append(dPred)
      
    #label selection
    if sTypeLabel=='on_off':
        dLabels=dOnOff
    elif sTypeLabel=='tremor':
        dLabels=dTrem
    elif sTypeLabel=='dyskinesia':
        dLabels=dDys
    else:
        print('sTypeLabel undefined')
    
    
    #creation of the matrix containing prediction from all classifiers
    bEnter=1
    for k in lDicts[0]: #first dictionary will be the lead
        if k!='measurement_id':
            #print(k)
            #print(dID[k])
            if dLabels[k]!='NA':
                vLabels.append(float(np.asarray(dLabels[k]))) #true labels
                vParID.append(float(np.asarray(dID[k]))) #participant ID
                vPredIter[0,0]=float(np.asarray(lDicts[0][k])) #first predicted value
                for j in range(1, iNumFiles):
                    fPred=lDicts[j].get(k)
                if fPred:
                    vPredIter[0,j]=float(np.asarray(fPred))
                else:
                    print(['Unkwnown key:' + k])
                    vPredIter[0,j]=float(np.asarray(lDicts[0][k]))
                    
            if bEnter==1:
                mPredictions=vPredIter
                bEnter=0
            else:
                mPredictions=np.append(mPredictions,vPredIter,axis=0)
         
    print('mPredictions and vLabels')
    print(mPredictions.shape)
    print(len(vLabels))
    # Random forest training - regression
    
    #scores = cross_val_score(clf, mPredictions, vLabels, cv=15)
    #print('Cross-validation score:')
    #print(scores.mean())

    clf=GradientBoostingRegressor(n_estimators=iEstimators, learning_rate=rLR, max_depth=iMD,\
                                      random_state=iRS, loss='ls').fit(mPredictions, vLabels)
    vNewPred=clf.predict(mPredictions)
    vRes1=mPredictions[:,[0]]
    vRes2=mPredictions[:,[1]]
    
    # we will include the testing data here
    return (vNewPred, vParID, vLabels, vRes1, vRes2, clf)
#print(mPredictions)

In [8]:
def randomForest_predTest(lTestPred, clf, sDirOut,sName):
    
    lDicts=[] #Will contain the two test predictions
    lmesID=[]
    iNumFiles=len(lTestPred)
    vPredIter=np.zeros((1,iNumFiles))

    for sFilePred in lTestPred:
        with open(sFilePred, mode='r') as infile:
            reader = csv.reader(infile)
            dPred = {rows[0]:rows[1] for rows in reader} #Prediction from the different classifiers (ivec-svr, boost...)
        lDicts.append(dPred)
      
    bEnter=1
    for k in lDicts[0]: #first dictionary will be the lead
        if k!='measurement_id':
            #print(k)
            #print(dID[k])
            lmesID.append(k)
            #vParID.append(float(np.asarray(dID[k]))) #participant ID
            vPredIter[0,0]=float(np.asarray(lDicts[0][k])) #first predicted value
            for j in range(1, iNumFiles):
                fPred=lDicts[j].get(k)
            if fPred:
                vPredIter[0,j]=float(np.asarray(fPred))
            else:
                print(['Unkwnown key:' + k])
                vPredIter[0,j]=float(np.asarray(lDicts[0][k]))

            if bEnter==1:
                mPredictions=vPredIter
                bEnter=0
            else:
                mPredictions=np.append(mPredictions,vPredIter,axis=0)
         
    
    vTestPred=clf.predict(mPredictions)
    df = pd.DataFrame({'measurement_id': lmesID, sName:vTestPred})
    df.to_csv(sDirOut+'submissionCisPD'+sName+'.csv', index=False)
    return(lmesID, vTestPred)



### Dyskinesia

In [8]:
#sFilePred1='/export/b15/nchen/BeatPD/cispd.kfold_prediction_on_off.csv'
sFilePred1='/export/b15/nchen/BeatPD/new_features/submission/cis-pd.dyskinesia.csv'
sFilePred2='/export/b15/nchen/BeatPD/new_features/submission/cis-pd.dyskinesia.csv'
sTypeLabel='on_off'
n_estimators=300
rLR=0.1
iMD=1
iRS=15
nudge=100
sFileLabels=sFilePred1

lFilesPred=[sFilePred1,sFilePred2];

vPredictions, vParID, vTrueLabels, vRes1,\
vRes2, clf = RandomForest_fusion (lFilesPred, sFileLabels, n_estimators, sTypeLabel, nudge, rLR, iMD,iRS)

print('Overall MSE Classif. 1 : ', get_final_score(vTrueLabels,np.array(vParID).astype(int), vRes1))
print('Overall MSE Classif. 2 : ', get_final_score(vTrueLabels, vParID, vRes2))
print('Overall MSE Fusion : ', get_final_score(vPredCross, vParIDCross,vTrueCross))




sTestPred1='/export/b15/nchen/BeatPD/new_features/submission/cis-pd.dyskinesia.csv'
sTestPred2='/export/c08/lmorove1/kaldi/egs/beatPDivec/on_off_noinact_auto30/exp/ivec_450/resiVecSVR_Fold/On_off_testing.csv'
lTestPred=[sTestPred1,sTestPred2]

sDirOut='/'
sName='on_off'
#lmesID, vTestPred = randomForest_predTest(lTestPred, clf, sDirOut, sName)
#print(vTestPred)




IndexError: list index out of range

### Tremor - Submission 2

In [5]:
# tsfresh predictions file 
sFilePred1='/export/b15/nchen/BeatPD/new_features/kfold_prediction_tremor.csv'

# SVR predictions file 
sFilePred2='/export/c08/lmorove1/kaldi/egs/beatPDivec/trem_noinact_auto30/exp/ivec_450/resiVecSVR_Fold_all/objs_450_kernel_linear_c_0.02_eps_0.1.csv'

# subchallenge 
sTypeLabel='tremor'

# RandomForest hyperparameter 
n_estimators=5

# Labels
sFileLabels='/export/b18/sjoshi/BeatPD/data/BeatPD/cis-pd.data_labels/CIS-PD_Training_Data_IDs_Labels.csv'


lFilesPred=[sFilePred1,sFilePred2];

vPredictions, vParID, vTrueLabels, vRes1, vRes2, clf = RandomForest_fusion(lFilesPred,
                                                                           sFileLabels,
                                                                           n_estimators,
                                                                           sTypeLabel)

sTestPred1='/export/b15/nchen/BeatPD/submission/cis-pd.tremor.csv'
sTestPred2='/export/c08/lmorove1/kaldi/egs/beatPDivec/trem_noinact_auto30/exp/ivec_450/resiVecSVR_Fold/Tremor_testing.csv'
lTestPred=[sTestPred1,sTestPred2]

sDirOut='/export/c08/lmorove1/kaldi/egs/beatPDivec/jupyternotebooks/submissionFiles/'
sName='tremor'
lmesID, vTestPred = randomForest_predTest(lTestPred, clf, sDirOut, sName)
print(vTestPred)



NameError: name 'RandomForest_fusion' is not defined

### Dyskinesia

In [ ]:
#sFilePred1='/export/b15/nchen/BeatPD/cispd.kfold_prediction_dyskinesia.csv'
sFilePred1='/export/b15/nchen/BeatPD/new_features/kfold_prediction_dyskinesia.csv'
sFilePred2='/export/c08/lmorove1/kaldi/egs/beatPDivec/v1_dysk_auto/exp/ivec_500/resiVecSVR_Fold_all/objs_500_kernel_linear_c_0.002_eps_0.1.csv'
sTypeLabel='dyskinesia'
n_estimators=5
sFileLabels='/export/b18/sjoshi/BeatPD/data/BeatPD/cis-pd.data_labels/CIS-PD_Training_Data_IDs_Labels.csv'

lFilesPred=[sFilePred1,sFilePred2];

vPredictions, vParID, vTrueLabels, vRes1, vRes2, clf = RandomForest_fusion (lFilesPred, sFileLabels, n_estimators, sTypeLabel)

sTestPred1='/export/b15/nchen/BeatPD/submission/cis-pd.dyskinesia.csv'
sTestPred2='/export/c08/lmorove1/kaldi/egs/beatPDivec/v1_dysk_auto/exp/ivec_500/resiVecSVR_Fold/Dyskinesia_testing.csv'
lTestPred=[sTestPred1,sTestPred2]

sDirOut='/export/c08/lmorove1/kaldi/egs/beatPDivec/jupyternotebooks/submissionFiles/'
sName='z_dyskinesia'
lmesID, vTestPred = randomForest_predTest(lTestPred, clf, sDirOut, sName)
print(vTestPred)



## Submission 4 - Average of predictions 